In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 📊 Rapport Codir Automatisé\n",
    "Ce notebook charge les données du fichier CSV, génère un graphique de synthèse commerciale, produit une synthèse automatique avec GPT et exporte un rapport PDF.\n",
    "\n",
    "**Pré-requis :**\n",
    "- Fichier `.env` à la racine avec :\n",
    "  ```\n",
    "  OPENAI_API_KEY=ta_cle\n",
    "  SLACK_BOT_TOKEN=ton_token\n",
    "  SLACK_CHANNEL_ID=ton_id\n",
    "  ```\n",
    "- Fichier `outputs/tables/data_codir.csv` existant avec les colonnes :\n",
    "  `Mois`, `CA réalisé (€)`, `Objectif (€)`, `Pipe (€)`, `Opportunités`, `Risques/Blocages`\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 1. Installer dépendances (exécuter une seule fois)\n",
    "!pip install openai matplotlib pandas slack_sdk fpdf python-dotenv\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 2. Imports\n",
    "import os\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "from slack_sdk import WebClient\n",
    "from fpdf import FPDF\n",
    "from dotenv import load_dotenv\n",
    "import openai\n",
    "\n",
    "# Charger variables d'environnement\n",
    "load_dotenv()\n",
    "OPENAI_API_KEY = os.getenv(\"OPENAI_API_KEY\")\n",
    "SLACK_BOT_TOKEN = os.getenv(\"SLACK_BOT_TOKEN\")\n",
    "SLACK_CHANNEL_ID = os.getenv(\"SLACK_CHANNEL_ID\")\n",
    "\n",
    "openai.api_key = OPENAI_API_KEY\n",
    "\n",
    "# Création des dossiers outputs si non existants\n",
    "os.makedirs(\"outputs/artifacts\", exist_ok=True)\n",
    "os.makedirs(\"outputs/reports\", exist_ok=True)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 3. Charger les données\n",
    "df = pd.read_csv(\"outputs/tables/data_codir.csv\")\n",
    "display(df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 4. Générer le graphique\n",
    "mois = df[\"Mois\"]\n",
    "ca = df[\"CA réalisé (€)\"]\n",
    "objectif = df[\"Objectif (€)\"]\n",
    "pipe = df[\"Pipe (€)\"]\n",
    "\n",
    "plt.figure(figsize=(8,5))\n",
    "plt.plot(mois, ca, marker='o', label='CA Réalisé')\n",
    "plt.plot(mois, objectif, marker='o', linestyle='--', label='Objectif')\n",
    "plt.bar(mois, pipe, alpha=0.2, label='Pipe')\n",
    "plt.title(\"Synthèse Commerciale Trimestrielle\")\n",
    "plt.ylabel(\"Montant (€)\")\n",
    "plt.legend()\n",
    "plt.tight_layout()\n",
    "graph_path = \"outputs/artifacts/ventes_graph.png\"\n",
    "plt.savefig(graph_path)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 5. Génération synthèse Codir avec GPT\n",
    "prompt = f\"\"\"\n",
    "Voici le tableau des indicateurs commerciaux du trimestre :\n",
    "Mois : {', '.join(df['Mois'])}\n",
    "CA réalisé : {', '.join(str(x) for x in df['CA réalisé (€)'])}\n",
    "Objectif : {', '.join(str(x) for x in df['Objectif (€)'])}\n",
    "Pipe : {', '.join(str(x) for x in df['Pipe (€)'])}\n",
    "Opportunités : {', '.join(df['Opportunités'])}\n",
    "Risques : {', '.join(df['Risques/Blocages'])}\n",
    "\n",
    "Donne-moi :\n",
    "1. Une synthèse Codir (en 3 phrases)\n",
    "2. Les 2 points de vigilance et opportunités majeures\n",
    "3. 2 actions prioritaires à piloter ce trimestre\n",
    "\"\"\"\n",
    "\n",
    "response = openai.chat.completions.create(\n",
    "    model=\"gpt-4o\",\n",
    "    messages=[{\"role\": \"user\", \"content\": prompt}],\n",
    "    max_tokens=400,\n",
    "    temperature=0.2,\n",
    ")\n",
    "synthese_codir = response.choices[0].message.content\n",
    "print(synthese_codir)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 6. Générer le PDF\n",
    "pdf_path = \"outputs/reports/rapport_codir.pdf\"\n",
    "pdf = FPDF()\n",
    "pdf.add_page()\n",
    "pdf.set_font('Arial', 'B', 16)\n",
    "pdf.cell(0, 10, \"Synthèse Codir Trimestrielle\", ln=True, align='C')\n",
    "pdf.ln(10)\n",
    "pdf.set_font('Arial', '', 12)\n",
    "pdf.multi_cell(0, 8, synthese_codir)\n",
    "pdf.ln(5)\n",
    "pdf.image(graph_path, x=10, w=180)\n",
    "pdf.output(pdf_path)\n",
    "print(f\"✅ Rapport PDF généré : {pdf_path}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 7. Envoi sur Slack (optionnel)\n",
    "if SLACK_BOT_TOKEN and SLACK_CHANNEL_ID:\n",
    "    slack_client = WebClient(token=SLACK_BOT_TOKEN)\n",
    "    try:\n",
    "        slack_client.chat_postMessage(\n",
    "            channel=SLACK_CHANNEL_ID,\n",
    "            text=f\"Synthèse Codir automatique :\\n{synthese_codir}\"\n",
    "        )\n",
    "        slack_client.files_upload_v2(\n",
    "            channel=SLACK_CHANNEL_ID,\n",
    "            file=graph_path,\n",
    "            title=\"Graphique Codir\"\n",
    "        )\n",
    "        slack_client.files_upload_v2(\n",
    "            channel=SLACK_CHANNEL_ID,\n",
    "            file=pdf_path,\n",
    "            title=\"Rapport PDF Codir\"\n",
    "        )\n",
    "        print(\"✅ Envoi Slack réussi\")\n",
    "    except Exception as e:\n",
    "        print(f\"❌ Erreur Slack : {e}\")\n",
    "else:\n",
    "    print(\"⚠️ Clés Slack manquantes, envoi ignoré.\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined